In [1]:
import sys

sys.path.append('../../src')
from pjplan import Task, WBS, DefaultScheduler, Resource

In [2]:
# Sample project
with WBS() as prj:
    prj // Task(1, 'Task 1', estimate=40, resource='Tester')
    prj // Task(2, 'Task 2', predecessors=[prj[1]], estimate=20, resource='Tester')
    with prj // Task(3, 'Task 3') as t:
        t // Task(4, 'Task 4', predecessors=[prj[2]], estimate=100)
        t // Task(5, 'Task 5', predecessors=[prj[2]], estimate=50)
    with prj // Task(6, 'Task 6') as t:
        t // Task(7, 'Task 7', predecessors=[prj[5]], estimate=16)
        t // Task(8, 'Task 8', predecessors=[prj[7]], estimate=16)

### Simple print

In [3]:
# Print full WBS
prj # or prj.print()

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1     Tester                     -      -    []           
 2   Task 2     Tester                     -      -    [1]          
 3   Task 3     -                          -      -    []           
 4      Task 4  -                          -      -    [2]          
 5      Task 5  -                          -      -    [2]          
 6   Task 6     -                          -      -    []           
 7      Task 7  -                          -      -    [5]          
 8      Task 8  -                          -      -    [7]          

In [4]:
# Print single task
prj[3]

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 3   Task 3     -                          -      -    []           
 4      Task 4  -                          -      -    [2]          
 5      Task 5  -                          -      -    [2]          

In [5]:
# Print children
prj[3].children

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 4   Task 4  -                          -      -    [2]          
 5   Task 5  -                          -      -    [2]          

In [6]:
# Print selected tasks
prj.tasks(resource='Tester')

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1  Tester                     -      -    []           
 2   Task 2  Tester                     -      -    [1]          

In [7]:
# Print predecessors/successors
prj[4].predecessors

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 2   Task 2  Tester                     -      -    [1]          

### Print selected fields

In [8]:
prj.print(['id', 'name', 'resource'])

 ID  NAME       RESOURCE 
 1   Task 1     Tester   
 2   Task 2     Tester   
 3   Task 3     -        
 4      Task 4  -        
 5      Task 5  -        
 6   Task 6     -        
 7      Task 7  -        
 8      Task 8  -        


### Hide children

In [9]:
prj.print(children=False)

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1  Tester                     -      -    []           
 2   Task 2  Tester                     -      -    [1]          
 3   Task 3  -                          -      -    []           
 6   Task 6  -                          -      -    []           


### Color themes

In [10]:
# See https://en.wikipedia.org/wiki/ANSI_escape_code for coloring details
theme = {
    'header_color': '94m', #blue
    'level_colors': ['91m', '92m', '93m']
}
prj.print(theme=theme)

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1     Tester                     -      -    []           
 2   Task 2     Tester                     -      -    [1]          
 3   Task 3     -                          -      -    []           
 4      Task 4  -                          -      -    [2]          
 5      Task 5  -                          -      -    [2]          
 6   Task 6     -                          -      -    []           
 7      Task 7  -                          -      -    [5]          
 8      Task 8  -                          -      -    [7]          


### Custom task colors

In [11]:
# See https://en.wikipedia.org/wiki/ANSI_escape_code for coloring details
prj.tasks(resource='Tester').print_color='30:41m'
prj

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
[30:41m 1  [30:41m Task 1    [30:41m Tester   [30:41m          [30:41m       [30:41m -     [30:41m -   [30:41m []           
[30:41m 2  [30:41m Task 2    [30:41m Tester   [30:41m          [30:41m       [30:41m -     [30:41m -   [30:41m [1]          
 3   Task 3     -                          -      -    []           
 4      Task 4  -                          -      -    [2]          
 5      Task 5  -                          -      -    [2]          
 6   Task 6     -                          -      -    []           
 7      Task 7  -                          -      -    [5]          
 8      Task 8  -                          -      -    [7]          

# Visualize resource usage

In [12]:
# Let's create schedule
p = prj.clone()
p.tasks(resource=None).resource='Developer'
p.tasks.print_color = None

schedule = DefaultScheduler(resources=[Resource(name='Tester'), Resource(name='Developer')]).calc(p)
schedule.wbs

 ID  NAME       RESOURCE   ESTIMATE  SPENT  START             END               PREDECESSORS 
 1   Task 1     Tester                      10.04.2023 00:00  15.04.2023 00:00  []           
 2   Task 2     Tester                      17.04.2023 00:00  19.04.2023 12:00  [1]          
 3   Task 3     Developer                   19.04.2023 00:00  15.05.2023 18:00  []           
 4      Task 4  Developer                   19.04.2023 00:00  05.05.2023 12:00  [2]          
 5      Task 5  Developer                   05.05.2023 12:00  15.05.2023 18:00  [2]          
 6   Task 6     Developer                   15.05.2023 18:00  19.05.2023 18:00  []           
 7      Task 7  Developer                   15.05.2023 18:00  17.05.2023 18:00  [5]          
 8      Task 8  Developer                   17.05.2023 18:00  19.05.2023 18:00  [7]          

In [13]:
# Visualize usage table
schedule.resource_usage

| DATE     | DEVELOPER | TESTER |
| 23-04-10 | 0.0       | 8.0    |
| 23-04-11 | 0.0       | 8.0    |
| 23-04-12 | 0.0       | 8.0    |
| 23-04-13 | 0.0       | 8.0    |
| 23-04-14 | 0.0       | 8.0    |
| 23-04-15 | 0.0       | 0.0    |
| 23-04-16 | 0.0       | 0.0    |
| 23-04-17 | 0.0       | 8.0    |
| 23-04-18 | 0.0       | 8.0    |
| 23-04-19 | 8.0       | 4.0    |
| 23-04-20 | 8.0       | 0.0    |
| 23-04-21 | 8.0       | 0.0    |
| 23-04-22 | 0.0       | 0.0    |
| 23-04-23 | 0.0       | 0.0    |
| 23-04-24 | 8.0       | 0.0    |
| 23-04-25 | 8.0       | 0.0    |
| 23-04-26 | 8.0       | 0.0    |
| 23-04-27 | 8.0       | 0.0    |
| 23-04-28 | 8.0       | 0.0    |
| 23-04-29 | 0.0       | 0.0    |
| 23-04-30 | 0.0       | 0.0    |
| 23-05-01 | 8.0       | 0.0    |
| 23-05-02 | 8.0       | 0.0    |
| 23-05-03 | 8.0       | 0.0    |
| 23-05-04 | 8.0       | 0.0    |
| 23-05-05 | 8.0       | 0.0    |
| 23-05-06 | 0.0       | 0.0    |
| 23-05-07 | 0.0       | 0.0    |
| 23-05-08 | 8